# State Questionnaire Analysis

## Imports and Global Helper Functions

In [ ]:
import warnings
import json
from pathlib import Path

import pandas as pd
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, colors_dark, register_fausans_font
import biopsykit as bp

from stresspose_analysis.datasets.pilotstudy import PilotStudyDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()

plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
root_path = Path("../..").resolve()

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])
dataset = PilotStudyDataset(base_path)

In [ ]:
output_path = root_path.joinpath("results")
img_path = output_path.joinpath("plots")
stats_path = output_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([img_path, stats_path])

In [ ]:
# dict for renaming condition names for use in plots
condition_mapping = {"tsst": "TSST", "ftsst": "f-TSST"}

In [ ]:
quest_data = dataset.questionnaire
quest_data = quest_data.join(dataset.condition_first).join(dataset.cortisol_non_responder)
quest_data = quest_data.set_index(["condition_first", "non_responder"], append=True)
quest_data.head()

## PANAS

Get differences in negative affect change ($\text{PANAS}_{post} - \text{PANAS}_{pre}$) between conditions (TSST vs. f-TSST)

In [ ]:
panas_time = bp.questionnaires.utils.wide_to_long(
    quest_data.filter(like="PANAS"), quest_name="PANAS", levels=["subscale", "condition", "time"]
)
panas_time = panas_time.rename(condition_mapping, level="condition").xs("NegativeAffect", level="subscale")
panas_time.head()

In [ ]:
panas_data = dataset.panas_diff
panas_data = panas_data.rename(condition_mapping, level="condition").xs("NegativeAffect", level="subscale")
panas_data.head()

### PANAS Response (pre-post)

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True)
axs = iter(axs)

for key, df in panas_time.groupby("condition"):
    ax = next(axs)
    pg.plot_paired(data=df.reset_index(), dv="PANAS", within="time", subject="subject", order=["pre", "post"], ax=ax)
    ax.set_title(key)
    ax.set_ylabel("PANAS Negative Affect Scale [AU]")

fig.tight_layout()

In [ ]:
steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "PANAS",
    "within": "time",
    "subject": "subject",
    "parametric": False,
    "groupby": "condition",
    "multicomp": {"method": "bonf", "levels": True},
}

pipeline_panas_time = bp.stats.StatsPipeline(steps, params)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    pipeline_panas_time.apply(panas_time)
pipeline_panas_time.display_results()
pipeline_panas_time.export_statistics(stats_path.joinpath("stats_panas_pre_post.xlsx"))

### PANAS Difference (f-)TSST

In [ ]:
steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "PANAS",
    "within": "condition",
    "subject": "subject",
    "parametric": False,
    "multicomp": {"method": "bonf", "levels": True},
}

pipeline_panas = bp.stats.StatsPipeline(steps, params)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    pipeline_panas.apply(panas_data)
pipeline_panas.display_results()
pipeline_panas.export_statistics(stats_path.joinpath("stats_panas_response.xlsx"))

### Plots

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

box_pairs, pvalues = pipeline_panas.sig_brackets("test", "within", plot_type="single", x="condition")

bp.plotting.feature_boxplot(
    panas_data,
    x="condition",
    y="PANAS",
    order=["f-TSST", "TSST"],
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    palette=cmaps.faculties_light,
    ax=ax,
)
ax.axhline(y=0, color=colors_dark.fau, ls="--", lw=2, zorder=-1)

ax.set_xlabel("Condition")
ax.set_ylabel("PANAS Negative Affect Change [AU]")
fig.tight_layout()

## STADI State Anxiety

In [ ]:
stadi_time = bp.questionnaires.utils.wide_to_long(
    quest_data.filter(like="STADI_State"), quest_name="STADI_State", levels=["subscale", "condition", "time"]
)
stadi_time = stadi_time.rename(condition_mapping, level="condition").xs("Anxiety", level="subscale")
stadi_time.head()

In [ ]:
stadi_data = dataset.stadi_state_diff
stadi_data = stadi_data.rename(condition_mapping, level="condition")
stadi_data = stadi_data.xs("Anxiety", level="subscale")

stadi_data.head()

### STADI Response (pre-post)

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True)
axs = iter(axs)

for key, df in stadi_time.groupby("condition"):
    ax = next(axs)
    pg.plot_paired(
        data=df.reset_index(), dv="STADI_State", within="time", subject="subject", order=["pre", "post"], ax=ax
    )
    ax.set_title(key)
    ax.set_ylabel("STADI Anxiety Scale [AU]")

fig.tight_layout()

In [ ]:
steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "STADI_State",
    "within": "time",
    "subject": "subject",
    "parametric": False,
    "groupby": "condition",
    "multicomp": {"method": "bonf", "levels": True},
}

pipeline_stadi_time = bp.stats.StatsPipeline(steps, params)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    pipeline_stadi_time.apply(stadi_time)
pipeline_stadi_time.display_results()
pipeline_stadi_time.export_statistics(stats_path.joinpath("stats_stadi_pre_post.xlsx"))

### STADI Difference (f-)TSST

In [ ]:
steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {"dv": "STADI_State", "within": "condition", "subject": "subject", "parametric": False}

pipeline_stadi = bp.stats.StatsPipeline(steps, params)
pipeline_stadi.apply(stadi_data)
pipeline_stadi.display_results()
pipeline_stadi.export_statistics(stats_path.joinpath("stats_stadi_response.xlsx"))

### Plots

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

box_pairs, pvalues = pipeline_stadi.sig_brackets("test", "within", plot_type="single")

bp.plotting.feature_boxplot(
    stadi_data,
    x="condition",
    y="STADI_State",
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    palette=cmaps.faculties_light,
    ax=ax,
)

ax.axhline(y=0, color=colors_dark.fau, ls="--", lw=2, zorder=-1)
ax.set_xlabel("Condition")
ax.set_ylabel("State Anxiety Change [AU]")
fig.tight_layout()

## Multi-Plot

In [ ]:
fig, axs = plt.subplots(figsize=(8, 4), ncols=2)

box_pairs, pvalues = pipeline_panas.sig_brackets("test", "within", plot_type="single")

bp.plotting.feature_boxplot(
    panas_data,
    x="condition",
    y="PANAS",
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    palette=cmaps.faculties_light,
    ax=axs[0],
)

box_pairs, pvalues = pipeline_stadi.sig_brackets("test", "within", plot_type="single")

bp.plotting.feature_boxplot(
    stadi_data,
    x="condition",
    y="STADI_State",
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    palette=cmaps.faculties_light,
    ax=axs[1],
)

for ax, title, quest in zip(axs, list("ab"), ["PANAS", "STADI"]):
    ax.axhline(y=0, color=colors_dark.fau, ls="--", lw=2, zorder=-1)
    ax.set_xlabel("Condition")
    ax.set_title(f"({title}) {quest}")

axs[0].set_ylabel("Negative Affect Change [AU]")
axs[1].set_ylabel("State Anxiety Change [AU]")
fig.tight_layout(w_pad=1.0)

fig.savefig(img_path.joinpath("img_selfreport_response_multiplot.pdf"), transparent=True)

## TeX Export

In [ ]:
export_dict = {
    "Negative Affect": pipeline_panas_time.results["pairwise_tests"],
    "Anxiety": pipeline_stadi_time.results["pairwise_tests"],
}
stats_results = pd.concat(export_dict, names=["dimension"])
stats_results

In [ ]:
print(
    pipeline_panas_time.results_to_latex_table(
        "pairwise_tests",
        data=stats_results,
        column_format="ll|S[table-format=2.1]SS",
        index_kws={"index_level_names_tex": ["Dimension", "Condition"]},
    )
)

In [ ]:
pipeline_stadi_time.stats_to_latex("pairwise_tests")

In [ ]:
pipeline_stadi.stats_to_latex("pairwise_tests")

In [ ]:
pipeline_panas_time.stats_to_latex("pairwise_tests")

In [ ]:
pipeline_panas.stats_to_latex("pairwise_tests")